Loading and converting keras to tensorflow pb

In [1]:
from keras import backend as K

Using TensorFlow backend.
C:\Users\Feng\Miniconda3\envs\aind-vui\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
K.set_learning_phase(0)

In [4]:
from keras.models import load_model
model = load_model('./model/keras_end_model.h5')


C:\Users\Feng\Miniconda3\envs\aind-vui\lib\site-packages\keras\models.py:258: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
print(model.outputs)
print(model.inputs)


[<tf.Tensor 'softmax_1/truediv:0' shape=(?, ?, 29) dtype=float32>]
[<tf.Tensor 'the_input_1:0' shape=(?, ?, 13) dtype=float32>]


In [6]:
import tensorflow as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

INFO:tensorflow:Froze 44 variables.
Converted 44 variables to const ops.


In [7]:
# Save to ./model/tf_model.pb
tf.train.write_graph(frozen_graph, "model", "model_end.pb", as_text=False)

'model\\model_end.pb'